## Cell 1: Import Libraries

In [11]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported!")

✅ Libraries imported!


## Cell 2: Load Dataset

In [12]:
df = pd.read_csv('data/cleaned/india_aq_cleaned_features.csv')
print(f"✅ Loaded: {len(df):,} records, {len(df.columns)} columns")

# Check missing values
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
print(f"\n⚠️ Columns with missing values:")
for col, count in missing.items():
    print(f"   {col}: {count:,} ({count/len(df)*100:.1f}%)")

✅ Loaded: 106,369 records, 84 columns

⚠️ Columns with missing values:
   power_plants_distance_m: 87,217 (82.0%)
   agricultural_distance_m: 72,965 (68.6%)
   dump_sites_distance_m: 65,727 (61.8%)
   commercial_distance_m: 47,099 (44.3%)
   industrial_distance_m: 30,196 (28.4%)
   residential_distance_m: 17,496 (16.4%)


## Cell 3: Handle Missing Values
**Strategy:**
- Distance columns (NaN = no feature found nearby) → Fill with large value (9999m)
- Area/Count columns (NaN = no feature) → Fill with 0
- Weather outliers → Clip to realistic ranges

In [13]:
# Distance columns: NaN means no feature found → use large distance
distance_cols = [c for c in df.columns if '_distance_m' in c]
for col in distance_cols:
    df[col] = df[col].fillna(9999)
print(f"✅ Filled {len(distance_cols)} distance columns with 9999m (far away)")

# Area/Count columns: NaN means no feature → use 0
area_cols = [c for c in df.columns if '_area_sqm' in c or '_count' in c]
for col in area_cols:
    df[col] = df[col].fillna(0)
print(f"✅ Filled {len(area_cols)} area/count columns with 0")

# Verify no missing values
remaining = df.isnull().sum().sum()
print(f"\n✅ Remaining missing values: {remaining}")

✅ Filled 7 distance columns with 9999m (far away)
✅ Filled 13 area/count columns with 0

✅ Remaining missing values: 0


## Cell 4: Handle Weather Outliers
Clip unrealistic weather values to valid ranges.

In [14]:
# Temperature: -10°C to 50°C (realistic for India)
before_temp = len(df[(df['temperature'] < -10) | (df['temperature'] > 50)])
df['temperature'] = df['temperature'].clip(-10, 50)
print(f"✅ Temperature: Clipped {before_temp:,} outliers to [-10, 50]°C")

# Humidity: 0% to 100%
before_hum = len(df[(df['humidity'] < 0) | (df['humidity'] > 100)])
df['humidity'] = df['humidity'].clip(0, 100)
print(f"✅ Humidity: Clipped {before_hum:,} outliers to [0, 100]%")

# Wind speed: 0 to 50 m/s
before_wind = len(df[(df['wind_speed'] < 0) | (df['wind_speed'] > 50)])
df['wind_speed'] = df['wind_speed'].clip(0, 50)
print(f"✅ Wind speed: Clipped {before_wind:,} outliers to [0, 50] m/s")

# Pollutants: Remove extreme outliers (>99.9 percentile)
pollutants = ['pm25', 'pm10', 'no2', 'co', 'so2', 'o3']
total_clipped = 0
for pol in pollutants:
    upper = df[pol].quantile(0.999)
    clipped = (df[pol] > upper).sum()
    df[pol] = df[pol].clip(0, upper)
    total_clipped += clipped
print(f"✅ Pollutants: Clipped {total_clipped:,} extreme values (>99.9 percentile)")

✅ Temperature: Clipped 265 outliers to [-10, 50]°C
✅ Humidity: Clipped 1,500 outliers to [0, 100]%
✅ Wind speed: Clipped 2,072 outliers to [0, 50] m/s
✅ Pollutants: Clipped 639 extreme values (>99.9 percentile)


## Cell 5: Select Essential Columns
Keep only columns needed for model training.

In [15]:
essential_cols = [
    # Location
    'state', 'district', 'location_id', 'location_name', 'latitude', 'longitude',
    # Time
    'datetime_ist', 'hour', 'day_of_week', 'month', 'season', 'is_weekend', 'is_rush_hour',
    # Pollutants
    'pm25', 'pm10', 'no2', 'co', 'so2', 'o3',
    # Weather
    'temperature', 'humidity', 'wind_speed',
    # Spatial (roads)
    'roads_distance_m', 'roads_count',
    # Spatial (industrial)
    'industrial_distance_m', 'industrial_area_sqm',
    # Spatial (agricultural)
    'agricultural_distance_m', 'agricultural_area_sqm',
    # Spatial (dump sites)
    'dump_sites_distance_m', 'dump_sites_count',
]

cols = [c for c in essential_cols if c in df.columns]
df_clean = df[cols].copy()

print(f"✅ Selected {len(df_clean.columns)} essential columns")
print(f"\nColumns: {list(df_clean.columns)}")

✅ Selected 30 essential columns

Columns: ['state', 'district', 'location_id', 'location_name', 'latitude', 'longitude', 'datetime_ist', 'hour', 'day_of_week', 'month', 'season', 'is_weekend', 'is_rush_hour', 'pm25', 'pm10', 'no2', 'co', 'so2', 'o3', 'temperature', 'humidity', 'wind_speed', 'roads_distance_m', 'roads_count', 'industrial_distance_m', 'industrial_area_sqm', 'agricultural_distance_m', 'agricultural_area_sqm', 'dump_sites_distance_m', 'dump_sites_count']


## Cell 6: Calculate Thresholds
Use percentile-based thresholds for pollutant levels.

In [16]:
thresholds = {}
for col in ['pm25', 'pm10', 'no2', 'co', 'so2', 'o3']:
    thresholds[col] = {
        'low': df_clean[col].quantile(0.25),
        'medium': df_clean[col].quantile(0.50),
        'high': df_clean[col].quantile(0.75),
        'very_high': df_clean[col].quantile(0.90)
    }

print("✅ Thresholds:")
for pol, vals in thresholds.items():
    print(f"   {pol}: low={vals['low']:.1f}, med={vals['medium']:.1f}, high={vals['high']:.1f}, v.high={vals['very_high']:.1f}")

✅ Thresholds:
   pm25: low=29.7, med=51.8, high=77.3, v.high=109.4
   pm10: low=63.0, med=101.7, high=147.5, v.high=206.0
   no2: low=5.9, med=14.3, high=25.4, v.high=47.9
   co: low=0.4, med=0.7, high=1.0, v.high=1.6
   so2: low=5.1, med=10.2, high=17.4, v.high=30.7
   o3: low=9.3, med=21.9, high=37.7, v.high=63.0


## Cell 7: Define Scoring Functions
Each source has a scoring function based on:
- Spatial proximity
- Pollutant levels
- Temporal patterns

In [17]:
def score_vehicular(row, th):
    """Vehicular: Roads + NO2 + Rush hour"""
    score = 0
    dist = row.get('roads_distance_m', 9999)
    if dist <= 50: score += 5
    elif dist <= 100: score += 4
    elif dist <= 200: score += 3
    elif dist <= 500: score += 2
    
    if row.get('roads_count', 0) > 200: score += 3
    elif row.get('roads_count', 0) > 100: score += 2
    
    no2 = row.get('no2', 0)
    if no2 > th['no2']['very_high']: score += 5
    elif no2 > th['no2']['high']: score += 3
    
    if row.get('is_rush_hour', 0) == 1: score += 2
    return score

def score_industrial(row, th):
    """Industrial: Factories + SO2 + Industrial area"""
    score = 0
    dist = row.get('industrial_distance_m', 9999)
    if dist <= 500: score += 4
    elif dist <= 1000: score += 3
    elif dist <= 2000: score += 2
    
    area = row.get('industrial_area_sqm', 0)
    if area > 5000000: score += 5
    elif area > 1000000: score += 4
    elif area > 100000: score += 2
    
    so2 = row.get('so2', 0)
    if so2 > th['so2']['very_high']: score += 5
    elif so2 > th['so2']['high']: score += 3
    
    if row.get('is_weekend', 1) == 0: score += 1
    return score

def score_agricultural(row, th):
    """Agricultural: Farmland + PM + Dry season"""
    score = 0
    dist = row.get('agricultural_distance_m', 9999)
    if dist <= 500: score += 4
    elif dist <= 1000: score += 3
    
    area = row.get('agricultural_area_sqm', 0)
    if area > 100000: score += 4
    elif area > 50000: score += 2
    
    if row.get('pm10', 0) > th['pm10']['high']: score += 3
    if row.get('pm25', 0) > th['pm25']['high']: score += 2
    
    if row.get('season', '') in ['post_monsoon', 'winter']: score += 3
    return score

def score_burning(row, th):
    """Burning: Dump sites + CO + Evening"""
    score = 0
    dist = row.get('dump_sites_distance_m', 9999)
    if dist <= 300: score += 5
    elif dist <= 500: score += 4
    elif dist <= 1000: score += 3
    
    if row.get('dump_sites_count', 0) > 0: score += 2
    
    co = row.get('co', 0)
    if co > th['co']['very_high']: score += 4
    elif co > th['co']['high']: score += 2
    
    hour = row.get('hour', 12)
    if 18 <= hour <= 23 or 0 <= hour <= 5: score += 2
    return score

def score_natural(row, th):
    """Natural: High PM ratio + Low gases + Wind"""
    score = 0
    pm25 = row.get('pm25', 1)
    pm10 = row.get('pm10', 0)
    
    if pm25 > 0:
        ratio = pm10 / pm25
        if ratio > 3: score += 5
        elif ratio > 2: score += 3
    
    if row.get('no2', 999) < th['no2']['medium']: score += 2
    if row.get('so2', 999) < th['so2']['medium']: score += 2
    if row.get('co', 999) < th['co']['medium']: score += 2
    
    if row.get('wind_speed', 0) > 5: score += 3
    return score

print("✅ Scoring functions defined!")

✅ Scoring functions defined!


## Cell 8: Apply Labels
Assign pollution source based on highest score.

In [18]:
def assign_label(row, th):
    scores = {
        'Vehicular': score_vehicular(row, th),
        'Industrial': score_industrial(row, th),
        'Agricultural': score_agricultural(row, th),
        'Burning': score_burning(row, th),
        'Natural': score_natural(row, th)
    }
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    top_source, top_score = sorted_scores[0]
    margin = top_score - sorted_scores[1][1]
    
    if margin >= 5: confidence = 'High'
    elif margin >= 3: confidence = 'Medium'
    else: confidence = 'Low'
    
    return top_source, confidence, top_score

print("⏳ Labeling... (1-2 min)")
results = [assign_label(row, thresholds) for _, row in df_clean.iterrows()]

df_clean['pollution_source'] = [r[0] for r in results]
df_clean['confidence'] = [r[1] for r in results]
df_clean['score'] = [r[2] for r in results]

print("\n✅ Label Distribution:")
for label, count in df_clean['pollution_source'].value_counts().items():
    print(f"   {label:15}: {count:>6,} ({count/len(df_clean)*100:>5.1f}%)")

⏳ Labeling... (1-2 min)

✅ Label Distribution:
   Vehicular      : 42,445 ( 39.9%)
   Industrial     : 23,372 ( 22.0%)
   Agricultural   : 21,389 ( 20.1%)
   Natural        : 12,061 ( 11.3%)
   Burning        :  7,102 (  6.7%)


## Cell 9: Validate Labels

In [19]:
print("📊 Average Pollutants by Source:")
print(df_clean.groupby('pollution_source')[['no2', 'so2', 'co', 'pm25', 'pm10']].mean().round(2))

print("\n📊 Confidence Distribution:")
for conf, count in df_clean['confidence'].value_counts().items():
    print(f"   {conf:8}: {count:>6,} ({count/len(df_clean)*100:>5.1f}%)")

📊 Average Pollutants by Source:
                    no2    so2    co   pm25    pm10
pollution_source                                   
Agricultural      11.01  12.37  0.68  51.65  105.23
Burning           12.11   7.58  1.11  61.53   94.80
Industrial        23.43  22.26  0.94  71.54  120.17
Natural           10.04   6.12  0.42  29.68   79.65
Vehicular         29.05  14.42  0.79  67.93  131.30

📊 Confidence Distribution:
   Low     : 60,050 ( 56.5%)
   Medium  : 26,418 ( 24.8%)
   High    : 19,901 ( 18.7%)


## Cell 10: Create Encoded Labels

In [20]:
label_map = {'Vehicular': 0, 'Industrial': 1, 'Agricultural': 2, 'Burning': 3, 'Natural': 4}
df_clean['source_encoded'] = df_clean['pollution_source'].map(label_map)

print("✅ Label Encoding:")
for label, code in label_map.items():
    count = (df_clean['pollution_source'] == label).sum()
    print(f"   {code} = {label:15} ({count:,})")

✅ Label Encoding:
   0 = Vehicular       (42,445)
   1 = Industrial      (23,372)
   2 = Agricultural    (21,389)
   3 = Burning         (7,102)
   4 = Natural         (12,061)


## Cell 11: Final Dataset Stats

In [21]:
final_cols = [
    'state', 'district', 'location_id', 'location_name', 'latitude', 'longitude',
    'datetime_ist', 'hour', 'day_of_week', 'month', 'season', 'is_weekend', 'is_rush_hour',
    'pm25', 'pm10', 'no2', 'co', 'so2', 'o3',
    'temperature', 'humidity', 'wind_speed',
    'roads_distance_m', 'roads_count',
    'industrial_distance_m', 'industrial_area_sqm',
    'agricultural_distance_m', 'agricultural_area_sqm',
    'dump_sites_distance_m', 'dump_sites_count',
    'pollution_source', 'source_encoded', 'confidence'
]

df_final = df_clean[[c for c in final_cols if c in df_clean.columns]].copy()

print("📊 FINAL DATASET SUMMARY")
print("="*50)
print(f"Total Records: {len(df_final):,}")
print(f"Total Columns: {len(df_final.columns)}")
print(f"States: {df_final['state'].nunique()}")
print(f"Districts: {df_final['district'].nunique()}")
print(f"Locations: {df_final['location_id'].nunique()}")
print(f"Missing Values: {df_final.isnull().sum().sum()}")
print(f"\nData Types:")
print(df_final.dtypes.value_counts())

📊 FINAL DATASET SUMMARY
Total Records: 106,369
Total Columns: 33
States: 19
Districts: 49
Locations: 49
Missing Values: 0

Data Types:
float64    17
int64       9
object      7
Name: count, dtype: int64


## Cell 12: Save Final Datasets

In [22]:
os.makedirs('data/final', exist_ok=True)

# Full dataset
df_final.to_csv('data/final/enviroscan_final_dataset.csv', index=False)
print(f"✅ Saved: data/final/enviroscan_final_dataset.csv ({len(df_final):,} records)")

# Training set (High + Medium confidence)
df_train = df_final[df_final['confidence'].isin(['High', 'Medium'])].copy()
df_train.to_csv('data/final/enviroscan_training_set.csv', index=False)
print(f"✅ Saved: data/final/enviroscan_training_set.csv ({len(df_train):,} records)")

# Label summary
summary = df_final.groupby(['pollution_source', 'confidence']).size().reset_index(name='count')
summary.to_csv('data/final/label_summary.csv', index=False)
print(f"✅ Saved: data/final/label_summary.csv")

print("\n" + "="*50)
print("🎉 DATASET PREPARATION COMPLETE!")
print("="*50)
print(f"\nFull Dataset: {len(df_final):,} records")
print(f"Training Set: {len(df_train):,} records ({len(df_train)/len(df_final)*100:.1f}%)")

✅ Saved: data/final/enviroscan_final_dataset.csv (106,369 records)
✅ Saved: data/final/enviroscan_training_set.csv (46,319 records)
✅ Saved: data/final/label_summary.csv

🎉 DATASET PREPARATION COMPLETE!

Full Dataset: 106,369 records
Training Set: 46,319 records (43.5%)
